# CSV to JSON-LD via Morph

ref: https://github.com/morph-kgc/morph-kgc/tree/main/examples 

## Imports

In [1]:
import morph_kgc
import pandas as pd
import json
import jbutils
import pyoxigraph

In [2]:
df = pd.read_csv("../inputs/GDSC_metadata.csv")

In [3]:
print(df.columns.values)

['ID' 'Title' 'Creator' 'Url' 'DOI' 'Publisher' 'Rights' 'License'
 'Restrictions' 'Coverage' 'Relations' 'Type' 'Date' 'Description'
 'Provenance' 'Subject' 'Language' 'Structure' 'Geometry' 'Bands'
 'Pixel Dimension' 'EPSG' 'Scale' 'Extent' 'Lineage' 'Process Step'
 'Processor' 'Point of Contact' 'Data Level' 'Label' 'Values' 'Attributes'
 'Color Ramp' 'Tile Zoom Levels' 'Collections' 'Analytic Function'
 'Analytic EPSG' 'Thumbnail' 'Service Definition' 'Sponsor' 'Status'
 'Rights Statement' 'Disclaimer' 'Service' 'Source' 'Parameters'
 'File Name' 'Extension' 'Download' 'Table Name' 'Format' 'Dependency'
 'Custom ETL' 'Custom Parameters' 'up' 'podID' 'Index Fields' 'Bash ETL'
 'SQL Transform' 'Last Updated' 'Update Frequency' 'Last Accessed'
 'Columns' 'ETL Documentation' 'Notes' 'TODO']


In [4]:
df.head(10)

,ID,Title,Creator,Url,DOI,Publisher,Rights,License,Restrictions,Coverage,...,Index Fields,Bash ETL,SQL Transform,Last Updated,Update Frequency,Last Accessed,Columns,ETL Documentation,Notes,TODO
0,1.0,Miami-Dade Home Owner's Loan Corporation (HOLC...,University of Richmond's Digital Scholarship Lab,https://services.arcgis.com/jIL9msH9OI208GCb/a...,NaN,University of Richmond's Digital Scholarship Lab,Public Domain,NaN,Use items owned by Esri in ArcGIS Online in co...,United States,...,--,TBD,TBD,2021-08-17,Never,--,OBJECTID|HOLC_grade|city|HOLC|Code|ST |CitySta...,Direct download as ESRI json from ESRI service...,NaN,update to reflect Miami only\ninclude other ci...
1,2.0,2019 Florida Census Tracts,Department of Commerce|U.S. Census Bureau|Geog...,https://www.census.gov/geographies/mapping-fil...,NaN,Department of Commerce|U.S. Census Bureau|Geog...,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2021-09-22,Never,--,"STATEFP(string,2)|COUNTYFP(string,3)|TRACTCE(s...",ogr2ogr shapefile to postGIS,NaN,NaN
2,3.0,2019 Miami-Dade ACS 5 Year Estimates - Tract L...,Department of Commerce|U.S. Census Bureau,https://www.census.gov/data/developers/data-se...,NaN,Department of Commerce|U.S. Census Bureau,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2021-06-02,Never,--,https://api.census.gov/data/2019/acs/acs5/vari...,The custom function acs_customgregate is given...,please use this as a template for all ACS esti...,NaN
3,4.0,2019 Florida ACS 5 Year Estimates - Tract Leve...,Department of Commerce|U.S. Census Bureau,https://www.census.gov/data/developers/data-se...,NaN,Department of Commerce|U.S. Census Bureau,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2022-11-22,Never,--,https://api.census.gov/data/2019/acs/acs5/vari...,The custom function acs_customgregate is given...,update using 2019 DVMT ACS as model,NaN
4,5.0,2020 Florida Census Tracts - Florida,Department of Commerce|U.S. Census Bureau|Geog...,https://www.census.gov/geographies/mapping-fil...,NaN,Department of Commerce|U.S. Census Bureau|Geog...,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2021-09-22,Never,--,"STATEFP(string,2)|COUNTYFP(string,3)|TRACTCE(s...",ogr2ogr shapefile to postGIS,NaN,NaN
5,6.0,2020 Florida ACS 5 Year Estimates - Tract Leve...,Department of Commerce|U.S. Census Bureau,https://www.census.gov/data/developers/data-se...,NaN,Department of Commerce|U.S. Census Bureau,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2021-06-02,Never,--,https://api.census.gov/data/2020/acs/acs5/vari...,The custom function acs_customgregate is given...,update using 2019 DVMT ACS as model,update to reflect Florida only\nimprove column...
6,7.0,2020 Florida ACS 5 Year Estimates - Tract Leve...,Department of Commerce|U.S. Census Bureau,https://www.census.gov/data/developers/data-se...,NaN,Department of Commerce|U.S. Census Bureau,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2022-11-22,Never,--,https://api.census.gov/data/2020/acs/acs5/vari...,The custom function acs_customgregate is given...,update using 2019 DVMT ACS as model,update to reflect Florida only\nimprove column...
7,8.0,2021 Florida Census Tracts - Florida,Department of Commerce|U.S. Census Bureau|Geog...,https://www.census.gov/geographies/mapping-fil...,NaN,Department of Commerce|U.S. Census Bureau|Geog...,Public Domain,This Software was created by U.S. Government e...,NaN,United States|Florida,...,--,TBD,TBD,2021-09-22,Never,--,"STATEFP(string,2)|COUNTYFP(string,3)|TRACTCE(s...",ogr2ogr shapefile to postGIS,NaN,NaN
8,9.0,2021 Miami-Dade ACS 5 Year Estimates - Tract L...,Department of Commerce|U.S. Census Bureau,https://www.census.gov/data/developers/data-se...,NaN,Department

In [5]:
# Select specific columns
selected_columns = ['Title', 'Geometry', 'Extent']
new_df = df[selected_columns]

fdf = new_df[new_df['Geometry'] != '--']  # Replace 'col1' with your actual column name

fdf2 = fdf.dropna(subset=['Extent'], inplace=False)  # Get the result without modifying df
fdf2 = fdf2.copy()  # Create a copy to avoid SettingWithCopyWarning

In [7]:
fdf2.head(10)

,Title,Geometry,Extent
0,Miami-Dade Home Owner's Loan Corporation (HOLC...,polygon,-122.767506653|25.705373207|-70.949200532|47.7...
1,2019 Florida Census Tracts,polygon,31.000968|24.396308|-87.634896|-79.974306
4,2020 Florida Census Tracts - Florida,polygon,31.000968|24.396308|-87.634896|-79.974306
7,2021 Florida Census Tracts - Florida,polygon,31.000968|24.396308|-87.634896|-79.974306
10,2021 Florida Census Block Groups - Florida,polygon,31.000968|24.396308|-87.634896|-79.974306
12,Florida State Funded Cleanup Sites,point,-115000.00244509568|-115000.0053524822|1014999...
15,"TIGER/Line Shapefile, 2022, state, Florida, Cu...",polygon,-87.634896|31.000968|-79.974306|24.396308
16,"TIGER/Line Shapefile, 2022, US, Current County",polygon,-179.231086|-14.601813|179.859681|71.439786
22,Miami-Dade Park Facilities,point,-8990813.946|2924852.2194|-8918818.8245|299584...
23,Miami-Dade Police Stations,point,-8952639.3862|2946045.317|-8922810.83|2989189....
